In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, Activation,concatenate, Attention, Bidirectional,GlobalAveragePooling1D
from keras import optimizers
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import LeakyReLU
import matplotlib.pyplot as plt
import keras_tuner as kt
import math
from datetime import datetime, timedelta
from scipy.stats import norm

In [2]:
start_date = '2016-01-01'
today = datetime.now()
end_date = datetime.strftime(today, '%Y-%m-%d')
#end_date = '2024-01-08'
print(end_date)
data = yf.download("ETH-USD", start=start_date, end=end_date) #Displays till yesterday as today is not complete
data = data.reset_index()
dates = data['Date']
data = data[['Close', 'High', 'Low', 'Volume']]
display(data)

2024-01-10


[*********************100%%**********************]  1 of 1 completed


,Close,High,Low,Volume
0,320.884003,329.451996,307.056000,893249984
1,299.252991,324.717987,294.541992,885985984
2,314.681000,319.453003,298.191986,842300992
3,307.907990,319.153015,298.513000,1613479936
4,316.716003,328.415009,307.024994,1041889984
...,...,...,...,...
2248,2268.647217,2276.764648,2209.537109,10860953290
2249,2241.624756,2271.359375,2219.781982,5970741680
2250,2222.865967,2257.127930,2211.562500,6490053615
2251,2333.032715,2358.815674,2171.993652,13830287095


In [3]:
Xscaler = MinMaxScaler()
Xdata = Xscaler.fit_transform(np.array(data))
Xdata.shape
Yscaler = MinMaxScaler()
Ydata = Yscaler.fit_transform(np.array([data['Close']]).transpose())

In [4]:
train_size = int(len(Xdata)*0.7)
test_size = len(Xdata) - train_size
Xtrain_data, Xtest_data = Xdata[0:train_size,:],Xdata[train_size:len(Xdata),:1]
Ytrain_data, Ytest_data = Ydata[0:train_size,:],Ydata[train_size:len(Xdata),:1]

In [5]:
def create_dataset(Xdata, Ydata, time_step = 1):
    dataX,dataY = [],[]
    for i in range(len(Xdata)-time_step-1):
                   a = Xdata[i:(i+time_step),0]
                   dataX.append(a)
                   dataY.append(Ydata[i + time_step,0])
    return np.array(dataX),np.array(dataY)

def build_timeseries(Xdata, Ydata, time_step):
    dim_0 = Xdata.shape[0] - time_step
    dim_1 = Xdata.shape[1]

    x = np.zeros((dim_0, time_step, dim_1))
    y = np.zeros((Ydata.shape[0] - time_step,))

    for i in range(dim_0):
        x[i] = Xdata[i:time_step+i]
        y[i] = Ydata[time_step+i]
    return x, y

In [6]:
time_step = 100
display(Ytrain_data.transpose()[0])
X_train, Y_train = build_timeseries(Xtrain_data, Ytrain_data.transpose()[0], time_step)
X_test, Y_test = build_timeseries(Xtest_data, Ytest_data.transpose()[0], time_step)

X_train = X_train.reshape((X_train.shape[0], time_step, X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], time_step, X_test.shape[2]))
display(X_train.shape)
display(Y_train.shape)

array([0.0500395 , 0.0454642 , 0.04872747, ..., 0.60616413, 0.58170245,
       0.53573732])

(1477, 100, 4)

(1477,)

In [7]:
model = Sequential()
model.add(LSTM(50,return_sequences = True,input_shape = (X_train.shape[1],X_train.shape[2])))
model.add(LSTM(50,return_sequences = True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss = 'mae',optimizer = 'adam')

ERROR with MODEL FIT IT WAS 3AM I NEED TO SLP

In [8]:
lstm_model = model.fit(X_train,Y_train,validation_data = (X_test,Y_test),epochs = 100,batch_size = 50, verbose = 1)

Epoch 1/100

29/30 [============================>.] - ETA: 0s - loss: 0.0484

ValueError: in user code:

    File "c:\Users\user\anaconda3\envs\MachineLearningEnv\Lib\site-packages\keras\src\engine\training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\user\anaconda3\envs\MachineLearningEnv\Lib\site-packages\keras\src\engine\training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\user\anaconda3\envs\MachineLearningEnv\Lib\site-packages\keras\src\engine\training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\user\anaconda3\envs\MachineLearningEnv\Lib\site-packages\keras\src\engine\training.py", line 1917, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\user\anaconda3\envs\MachineLearningEnv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\user\anaconda3\envs\M

In [ ]:
plt.plot(lstm_model.history['loss'], label='train')
plt.plot(lstm_model.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# make a prediction
yhat = model.predict(X_test)
# invert scaling for forecast
inv_yhat = Yscaler.inverse_transform(yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = Y_test.reshape((len(Y_test), 1))
inv_y = Yscaler.inverse_transform(test_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
look_back = 100
trainPredictPlot = np.empty_like(Ydata)
trainPredictPlot[:,:] = np.nan
trainPredictPlot[look_back : len(train_predict)+look_back,:] = train_predict
testPredictPlot = np.empty_like(Ydata)
testPredictPlot[:,:] = np.nan
testPredictPlot[len(train_predict)+(look_back)*2 + 1 : len(Ydata) - 1,:] = test_predict
plt.plot(Yscaler.inverse_transform(Ydata))
plt.plot(Yscaler.inverse_transform(trainPredictPlot))
plt.plot(Yscaler.inverse_transform(testPredictPlot))
plt.show()

In [ ]:
prediction_data = np.array(Xdata[-100:])
prediction_data = prediction_data.reshape((prediction_data.shape[0], 1, prediction_data.shape[1]))
display(prediction_data.shape)
prediction = Yscaler.inverse_transform(model.predict(prediction_data))[0][0]
print("Prediction for: ", end_date, "        CLOSE: ", prediction)